ksekinisame apo to baseline
meta skeftika na valo ta idia pou eixe gia abstracts kai gia tous authors
tora 8a prospa8iso na kano hyperparameter tuning kai na xrisimopoiiso ki alla modela
Meta 8a prospa8iso na xrisimopoiiso cosine similrity me tin xrisi to word2vec
to cosine similarity den doulevei giati ta vector exoun diaforetika sized kati prepei na allakso



In [70]:
import networkx as nx
import gensim
import csv
import numpy as np
from random import randint
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
import nltk
from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from collections import Counter
from nltk.corpus import stopwords

In [61]:
# Create a graph
G = nx.read_edgelist('edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)

Number of nodes: 138499
Number of edges: 1091955


In [62]:
# Read the abstract of each paper
abstracts = dict()
with open('abstracts.txt', 'r', encoding='utf-8') as f:
    for line in f:
        node, abstract = line.split('|--|')
        abstracts[int(node)] = abstract

In [63]:
# Read the authors of each paper
authors = dict()
with open('authors.txt', 'r', encoding='utf-8') as f:
    for line in f:
        node, author = line.split('|--|')
        authors[int(node)] = author

In [64]:
tokenized_authors = dict()
for node in authors:
    for word in authors[node].split(','):
        tokenized_authors[node] = [word for word in word_tokenize(word)]

In [131]:
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'])

In [132]:
tokenized_abstracts = dict()
for node in abstracts:
    tokenized_abstracts[node] = []
    for sent in sent_tokenize(abstracts[node]):
        for i in word_tokenize(sent):
            if i.lower() in stop_words:
                continue
            else:
                tokenized_abstracts[node].append(i.lower())

In [133]:
tokenized_abstracts[0]

['development',
 'automated',
 'system',
 'quality',
 'assessment',
 'aerodrome',
 'ground',
 'lighting',
 'agl',
 'accordance',
 'associated',
 'standards',
 'recommendations',
 'presented',
 'system',
 'composed',
 'image',
 'sensor',
 'placed',
 'inside',
 'cockpit',
 'aircraft',
 'record',
 'images',
 'agl',
 'normal',
 'descent',
 'aerodrome',
 'model-based',
 'methodology',
 'used',
 'ascertain',
 'optimum',
 'match',
 'template',
 'agl',
 'actual',
 'image',
 'data',
 'order',
 'calculate',
 'position',
 'orientation',
 'camera',
 'instant',
 'image',
 'acquired',
 'camera',
 'position',
 'orientation',
 'data',
 'used',
 'along',
 'pixel',
 'grey',
 'level',
 'imaged',
 'luminaire',
 'estimate',
 'value',
 'luminous',
 'intensity',
 'given',
 'luminaire',
 'compared',
 'expected',
 'brightness',
 'luminaire',
 'ensure',
 'operating',
 'required',
 'standards',
 'metric',
 'quality',
 'agl',
 'pattern',
 'determined',
 'experiments',
 'real',
 'image',
 'data',
 'presented',
 'd

In [134]:
# Map text to set of terms
for node in abstracts:
    abstracts[node] = set(abstracts[node].split(','))

In [135]:
# Map authors to set of terms
for node in authors:
    authors[node] = set(authors[node].split(','))

In [148]:
import math
epsilon = 1e-6
def counter_cosine_similarity(c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / ((magA * magB) + epsilon)

In [137]:
import gensim.models.keyedvectors as word2vec
model = word2vec.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [139]:
model2 = Word2Vec(tokenized_abstracts.values(), window=5, min_count=1, workers=4)

In [157]:
rank = nx.pagerank(G)

In [158]:
# its class label is 1 if it corresponds to an edge and 0, otherwise.
# Use the following 3 features for each pair of nodes:
# (1) sum of number of unique terms of the two nodes' abstracts
# (2) absolute value of difference of number of unique terms of the two nodes' abstracts
# (3) number of common terms between the abstracts of the two nodes
# (4) sum of number of unique terms of the two nodes' authors
# (5) absolute value of difference of number of unique terms of the two nodes' authors
# (6) number of common terms between the authors of the two nodes
# (7) cosine similarity between the abstracts of the two nodes

X_train = np.zeros((2*m, 9))
y_train = np.zeros(2*m)
n = G.number_of_nodes()
for i,edge in enumerate(G.edges()):
    # an edge
    X_train[i,0] = len(abstracts[edge[0]]) + len(abstracts[edge[1]])
    X_train[i,1] = abs(len(abstracts[edge[0]]) - len(abstracts[edge[1]]))
    X_train[i,2] = len(abstracts[edge[0]].intersection(abstracts[edge[1]]))
    X_train[i,3] = len(authors[edge[0]]) + len(authors[edge[1]])
    X_train[i,4] = abs(len(authors[edge[0]]) - len(authors[edge[1]]))
    X_train[i,5] = len(authors[edge[0]].intersection(authors[edge[1]]))
    X_train[i,6] = counter_cosine_similarity(Counter(tokenized_authors[edge[0]]), Counter(tokenized_authors[edge[1]]))
    X_train[i,7] = counter_cosine_similarity(Counter(tokenized_abstracts[edge[0]]), Counter(tokenized_abstracts[edge[1]]))
    X_train[i,8] = rank[edge[0]] + rank[edge[1]] # sum of ranks of the two nodes
    y_train[i] = 1

    # a randomly generated pair of nodes
    n1 = randint(0, n-1)
    n2 = randint(0, n-1)
    X_train[m+i,0] = len(abstracts[n1]) + len(abstracts[n2])
    X_train[m+i,1] = abs(len(abstracts[n1]) - len(abstracts[n2]))
    X_train[m+i,2] = len(abstracts[n1].intersection(abstracts[n2]))
    X_train[m+i,3] = len(authors[n1]) + len(authors[n2])
    X_train[m+i,4] = abs(len(authors[n1]) - len(authors[n2]))
    X_train[m+i,5] = len(authors[n1].intersection(authors[n2]))
    X_train[m+i,6] = counter_cosine_similarity(Counter(tokenized_authors[n1]), Counter(tokenized_authors[n2]))
    X_train[m+i,7] = counter_cosine_similarity(Counter(tokenized_abstracts[n1]), Counter(tokenized_abstracts[n2]))
    X_train[m+i,8] = rank[n1] + rank[n2] # sum of ranks of the two nodes
    y_train[m+i] = 0

In [159]:
print('Size of training matrix:', X_train.shape)

Size of training matrix: (2183910, 9)


In [160]:
# Read test data. Each sample is a pair of nodes
node_pairs = list()
with open('test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        node_pairs.append((int(t[0]), int(t[1])))

In [162]:
# Create the test matrix. Use the same 4 features as above
X_test = np.zeros((len(node_pairs), 9))
for i,node_pair in enumerate(node_pairs):
    X_test[i,0] = len(abstracts[node_pair[0]]) + len(abstracts[node_pair[1]])
    X_test[i,1] = abs(len(abstracts[node_pair[0]]) - len(abstracts[node_pair[1]]))
    X_test[i,2] = len(abstracts[node_pair[0]].intersection(abstracts[node_pair[1]]))
    X_test[i,3] = len(authors[node_pair[0]]) + len(authors[node_pair[1]])
    X_test[i,4] = abs(len(authors[node_pair[0]]) - len(authors[node_pair[1]]))
    X_test[i,5] = len(authors[node_pair[0]].intersection(authors[node_pair[1]]))
    X_test[i,6] = counter_cosine_similarity(Counter(tokenized_authors[node_pair[0]]), Counter(tokenized_authors[node_pair[1]]))
    X_test[i,7] = counter_cosine_similarity(Counter(tokenized_abstracts[node_pair[0]]), Counter(tokenized_abstracts[node_pair[1]]))
    X_test[i,8] = rank[node_pair[0]] + rank[node_pair[1]] # sum of ranks of the two nodes
print('Size of test matrix:', X_test.shape)

Size of test matrix: (106692, 9)


In [163]:
X_train, y_train = shuffle(X_train, y_train)

In [164]:
# Use logistic regression to predict if two nodes are linked by an edge
logreg = LogisticRegression(C=1e5, solver='lbfgs')
logreg.fit(X_train, y_train)


y_pred = logreg.predict_proba(X_test)
y_pred = y_pred[:,1]

C:\Users\ilsto\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [165]:
# Write predictions to a file
predictions = zip(range(len(y_pred)), y_pred)
with open("submissions.csv","w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for row in predictions:
        csv_out.writerow(row)